<a href="https://colab.research.google.com/github/jjandnn/fsgan/blob/master/fsgan_face_swapping_colab_%E4%B8%AD%E6%96%87_V2020_4_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/jjandnn/fsgan/blob/master/fsgan_face_swapping_colab_%E4%B8%AD%E6%96%87_V2020_4_25%EF%BC%89.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## FSGAN Face Swapping Demo
填写 [这份表单](https://docs.google.com/forms/d/e/1FAIpQLScyyNWoFvyaxxfyaPLnCIAxXgdxLEMwR9Sayjh3JpWseuYlOA/viewform?usp=sf_link),
收到邮件后，将 FSGAN 共享目录的快捷方式添加到你的谷歌云盘根目录中：

![Add to drive](https://raw.githubusercontent.com/wiki/YuvalNirkin/fsgan/media/add_to_drive.jpg)




特别鸣谢： 我们感谢 Dr. Eyal Gruss, [wangchao0899](https://github.com/wangchao0899)， [杜俊](https://github.com/jjandnn)， 和 [zhuhaozh](https://github.com/zhuhaozh) 对于这个 Demo 的帮助。

In [0]:
#@title 我们需要 Tesla P100 或 T4 的 GPU 来运行这个 demo，先运行本 cell 确认一下设备：
#@markdown 如果不是，请点击工具栏 “ 代码执行程序 -> 将该运行时恢复至出厂设置 “ ，然后重新连接。 
!nvidia-smi

Sun Apr 26 18:24:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#@title 挂载云盘
!rm -rf sample_data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 首次安装需运行

In [0]:
#@markdown 首次安装需要安装的部分，之后使用则不再需要运行本 cell

#@markdown 项目目录为： /content/drive/My Drive/fsganLab
!mkdir -p /content/drive/My\ Drive/fsganLab

%cd /content/drive/My Drive/fsganLab/
!git clone https://github.com/YuvalNirkin/face_detection_dsfd
!git clone https://github.com/YuvalNirkin/fsgan.git
!mkdir -p /content/drive/My\ Drive/fsganLab/data
!cp /content/drive/My\ Drive/fsganLab/fsgan/docs/examples/shinzo_abe.mp4 /content/drive/My\ Drive/fsganLab/data/source.mp4
!cp /content/drive/My\ Drive/fsganLab/fsgan/docs/examples/conan_obrien.mp4 /content/drive/My\ Drive/fsganLab/data/target.mp4
print ('基础程序已经安装，请勿重复安装')

基础程序已经安装，请勿重复安装


## 每次都需要运行的部分

In [0]:
import sys
from IPython.display import HTML, clear_output
from base64 import b64encode
import shutil

#@markdown 基础依赖的安装

# 安装所需要的依赖项
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

sys.path +=['/usr/local/lib/python3.7/site-packages','/content/drive/My Drive/fsganLab']
# 依赖，请勿修改安装顺序
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch -y
!conda install -c conda-forge yacs -y
!pip install opencv-python ffmpeg-python youtube-dl

# 定义音频编码器与显示视频的函数
import ffmpeg
def encode_audio(video_path, audio_path, output_path):
  ffmpeg.concat(ffmpeg.input(video_path), ffmpeg.input(audio_path), v=1, a=1) \
    .output(output_path, strict='-2').run(overwrite_output=True)


def display_video(video_path, width=640, clear=True):
  vid_data = open(video_path,'rb').read()
  vid_url = 'data:video/mp4;base64,' + b64encode(vid_data).decode()

  if clear:
    clear_output()

  return HTML(f"""
  <video width={width} controls>
    <source src={vid_url} type="video/mp4">
  </video>
  """)

delete_log = True #@param {type:"boolean"}
if delete_log == True:
  clear_output()
  print ('依赖已安装完毕')

依赖已安装完毕


In [0]:
import os
from fsgan.inference.swap import FaceSwapping
from fsgan.criterions.vgg_loss import VGGLoss

#@markdown 权重初始化，重启后需运行。每次改动参数，也需要再次运行。

#@markdown ---
#@markdown 权重文件夹的路径 （确保已经在您的云盘中，如果没有请点击填写 [这份表单](https://docs.google.com/forms/d/e/1FAIpQLScyyNWoFvyaxxfyaPLnCIAxXgdxLEMwR9Sayjh3JpWseuYlOA/viewform?usp=sf_link)）：
weights_dir = '/content/drive/My Drive/fsganLab/weights' #@param {type:"string"}
#@markdown 选择模型微调迭代（finetune iterations）的次数：
finetune_iterations = 800 #@param {type:"slider", min:100, max:2000, step:1}
#@markdown 打勾，则将嘴巴内部从分割中移除：
seg_remove_mouth = True #@param {type:"boolean"}
#@markdown ---


detection_model = os.path.join(weights_dir, 'v2/WIDERFace_DSFD_RES152.pth')
pose_model = os.path.join(weights_dir, 'shared/hopenet_robust_alpha1.pth')
lms_model = os.path.join(weights_dir, 'v2/hr18_wflw_landmarks.pth')
seg_model = os.path.join(weights_dir, 'v2/celeba_unet_256_1_2_segmentation_v2.pth')
reenactment_model = os.path.join(weights_dir, 'v2/nfv_msrunet_256_1_2_reenactment_v2.1.pth')
completion_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_inpainting_v2.pth')
blending_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_blending_v2.pth')
criterion_id_path = os.path.join(weights_dir, 'v2/vggface2_vgg19_256_1_2_id.pth')
criterion_id = VGGLoss(criterion_id_path)


face_swapping = FaceSwapping(
    detection_model=detection_model, pose_model=pose_model, lms_model=lms_model,
    seg_model=seg_model, reenactment_model=reenactment_model,
    completion_model=completion_model, blending_model=blending_model,
    criterion_id=criterion_id,
    finetune=True, finetune_save=True, finetune_iterations=finetune_iterations,
    seg_remove_mouth=finetune_iterations, batch_size=16, seg_batch_size=48,
    encoder_codec='mp4v')

delete_log = True #@param {type:"boolean"}
if delete_log == True:
    clear_output()
    print('权重初始化完毕')

权重初始化完毕


### 可选：从YouTube下载视频
您可以更改源视频的 URL。

选择开始和结束的时间，格式为：[hh:mm:ss]

处理后的文件存储默认路径为 ：/content/drive/My Drive/fsganLab/data/

文件名为 ：下表 used_for 选项的选择。 例，选 source ，则为 source.mp4



In [0]:
youtube_url = 'https://www.youtube.com/watch?v=5-s3ANu4eMs' #@param {type:"string"}
used_for = 'target' #@param ["source", "target"] {type:"string"}
cut_start = '00:01:40' #@param {type:"string"}
cut_end = '00:01:50' #@param {type:"string"}

shutil.rmtree('/content/drive/My Drive/fsganLab/data/'+ used_for)
!youtube-dl $youtube_url --merge-output-format mp4 -o /content/youtube_tmp.mp4
if used_for == 'source':
  !ffmpeg -y -i /content/youtube_tmp.mp4 -ss $cut_start -to $cut_end -r 25 /content/drive/My\ Drive/fsganLab/data/source.mp4
else:
  !ffmpeg -y -i /content/youtube_tmp.mp4 -ss $cut_start -to $cut_end -r 25 /content/drive/My\ Drive/fsganLab/data/target.mp4
!rm /content/youtube_tmp.mp4

display_video('/content/drive/My Drive/fsganLab/data/'+ used_for +'.mp4')


## 换脸

In [0]:
#@markdown ---
#@markdown 切换是否选择微调生成器：
finetune = True #@param {type:"boolean"}
#@markdown 填写源视频路径（ 直接复制左侧目录中的文件路径 ）：
source_path = '/content/drive/My Drive/fsganLab/data/source.mp4'#@param {type:"string"}
#@markdown 填写源视频采用的方案 [ "longest" | sequence number ]：
select_source = 'longest' #@param {type:"string"}
#@markdown 填写目标视频路径（ 直接复制左侧目录中的文件路径 ）：
target_path = '/content/drive/My Drive/fsganLab/data/target.mp4'#@param {type:"string"}
#@markdown 填写目标视频采用的方案 [ "longest" | sequence number ]：
select_target = 'longest' #@param {type:"string"}
#@markdown ---


output_tmp_path = '/content/drive/My Drive/fsganLab/data/output_tmp.mp4'
output_path = source_path.replace('.mp4','_to_') + target_path.replace('/content/drive/My Drive/fsganLab/data/','')
face_swapping(source_path, target_path, output_tmp_path,
              select_source, select_target, finetune)

# 编码音频并显示结果
encode_audio(output_tmp_path, target_path, output_path)
os.remove(output_tmp_path)
display_video(output_path)